In [8]:
import requests
import json
from urllib import parse # 한글
import time
import pandas as pd

input_dir='C:/Users/sherm/Desktop/workspace/likelion_dataton/data/'

api_key = "RGAPI-08f11c90-1ac2-496d-876c-ea5edddf6fc7" # 새로 발급받은 api_key
REQUEST_HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Whale/4.32.315.22 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

In [ ]:
import schedule, time
from scripts.main_pipeline import run_pipeline

def job():
    print("시작: 라이엇 데이터 수집")
    run_pipeline()
    print("완료!")

schedule.every().day.at("02:00").do(job)

while True:
    schedule.run_pending()
    time.sleep(60)


In [ ]:
def run_pipeline():
    users = load_user_list()            # 1. 소환사 리스트 불러오기
    for user in users:
        puuid = get_puuid(user)         # 2. 소환사 이름 → puuid
        match_ids = get_match_ids(puuid)  # 3. puuid → 최근 경기 목록
        for match_id in match_ids:
            if not is_already_saved(match_id):
                data = get_match_detail(match_id)  # 4. match 상세 데이터
                parsed = parse_match(data)         # 5. 데이터 평탄화
                insert_to_db(parsed)               # 6. DB에 저장


In [2]:
tiers=['challengerleagues','grandmasterleagues','masterleagues']
for tier in tiers:
    req_url=f'https://kr.api.riotgames.com/lol/league/v4/{tier}/by-queue/RANKED_SOLO_5x5'
    leauge_list = requests.get(req_url, headers=REQUEST_HEADERS).json()
    temp=leauge_list['entries']
    if tier == tiers[0]:
        chal_puuid = [temp[i]['puuid'] for i in range(len(temp))]
        print(f'챌린저 명부 완성 총 {len(chal_puuid)}명입니다')
    elif tier == tiers[1]:
        grand_puuid = [temp[i]['puuid'] for i in range(len(temp))]
        print(f'그마 명부 완성 총 {len(grand_puuid)}명입니다')
    else:
        master_puuid = [temp[i]['puuid'] for i in range(len(temp))]
        print(f'마스터 명부 완성 총 {len(master_puuid)}명입니다')
        
    print('5초동안 휴식')
    for i in range(5):
        time.sleep(1)
        print(i+1,end = ' ')
    print('\n')

print('임무 완료~')

챌린저 명부 완성 총 300명입니다
5초동안 휴식
1 2 3 4 5 

그마 명부 완성 총 700명입니다
5초동안 휴식
1 2 3 4 5 

마스터 명부 완성 총 10000명입니다
5초동안 휴식
1 2 3 4 5 

임무 완료~


In [ ]:
game_info_list=[]
req_url=f'https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
temp=requests.get(req_url, headers=REQUEST_HEADERS)
temp=temp.json()['entries']
challenger_puuid = [temp[i]['puuid'] for i in range(len(temp))]
print(f'챌린저 명부 완성 총 {len(master_puuid)}명입니다')
count=1

In [9]:
temp=pd.read_json(input_dir+'CHALLENGER_2025-07-20_matches.json')

In [21]:
len_temp=len(temp['metadata']) #['matchId']
game_list=[temp['metadata'][ordinal]['matchId'] for ordinal in range(len_temp)]

In [23]:
game_list

764

In [36]:
game_timeline_list=[]
for gameid in game_list:
    req_url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{gameid}/timeline'
    game_timeline_req = requests.get(req_url, headers=REQUEST_HEADERS)
    if game_timeline_req.status_code != 200: break
    print(f'상태코드 : {game_timeline_req.status_code}')
    game_timeline = game_timeline_req.json()
    game_timeline_list.append(game_timeline)
    
    print(f"gameID : {gameid}")
    print('10초동안 휴식')
    for i in range(10):
        time.sleep(1)
        print(i+1,end = ' ')
    print('\n')

print('임무 완료~')

상태코드 : 200
gameID : KR_7721738512
10초동안 휴식
1 2 3 4 5 6 7 8 9 10 

상태코드 : 200
gameID : KR_7720296596
10초동안 휴식
1 2 3 4 5 6 7 8 9 10 

상태코드 : 200
gameID : KR_7721350182
10초동안 휴식
1 2 3 4 5 6 7 8 9 10 

상태코드 : 200
gameID : KR_7720487398
10초동안 휴식
1 2 3 4 5 6 7 8 9 10 

상태코드 : 200
gameID : KR_7720408386
10초동안 휴식
1 2 3 4 5 6 7 8 9 10 

상태코드 : 200
gameID : KR_7720342573
10초동안 휴식
1 2 3 4 5 6 7 8 9 10 

상태코드 : 200
gameID : KR_7720272386
10초동안 휴식
1 2 3 4 5 6 7 8 9 10 

상태코드 : 200
gameID : KR_7721821501
10초동안 휴식
1 2 3 4 5 6 7 8 9 10 

상태코드 : 200
gameID : KR_7720598247
10초동안 휴식
1 2 3 4 5 6 7 8 9 10 

상태코드 : 200
gameID : KR_7720348477
10초동안 휴식
1 2 3 4 5 6 7 8 9 10 

상태코드 : 200
gameID : KR_7720467095
10초동안 휴식
1 2 3 4 5 6 7 8 9 10 

상태코드 : 200
gameID : KR_7720555971
10초동안 휴식
1 2 3 4 5 6 7 8 9 10 

상태코드 : 200
gameID : KR_7720272386
10초동안 휴식
1 2 3 4 5 6 7 8 9 10 

상태코드 : 200
gameID : KR_7721675219
10초동안 휴식
1 2 3 4 5 6 7 8 9 10 

상태코드 : 200
gameID : KR_7721174058
10초동안 휴식
1 2 3 4 5 6 7 8 9 10 

상태코드 : 200

In [42]:
with open('challenger_timeline.json','w', encoding="utf-8") as f:
    json.dump(game_timeline_list,f)

In [46]:
with open('challenger_timeline.json','r', encoding="utf-8") as f:
    temp=json.load(f)

In [ ]:
temp[0]

In [20]:
game_info_list=[]
tier='masterleagues'
req_url=f'https://kr.api.riotgames.com/lol/league/v4/{tier}/by-queue/RANKED_SOLO_5x5'
temp=requests.get(req_url, headers=REQUEST_HEADERS)
temp=temp.json()['entries']
master_puuid = [temp[i]['puuid'] for i in range(len(temp))]
print(f'마스터 명부 완성 총 {len(master_puuid)}명입니다')
count=1



for puuid in master_puuid:
    req_url = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?startTime={20250720}&endTime={20250723}&type=ranked&start={0}&count={2}'
    game_list = requests.get(req_url, headers=REQUEST_HEADERS).json()
    print('10초동안 휴식')
    for i in range(10):
        time.sleep(1)
        print(i+1,end = ' ')
    print('\n')
    
    for gameid in game_list:
        req_url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{gameid}/timeline'
        game_info = requests.get(req_url, headers=REQUEST_HEADERS).json()
        game_info_list.append(game_info)
    
    if count % 1000 == 0: pring(count)
    print(gameid)
    print('10초동안 휴식')
    for i in range(10):
        time.sleep(1)
        print(i+1,end = ' ')
    print('\n')

print('임무 완료~')

마스터 명부 완성 총 10000명입니다
10초동안 휴식
1 2 3 4 5 6 7 8 9 10 



NameError: name 'gameid' is not defined

In [25]:
req_url = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{'K1ScdaDuDBEhat0xMp7FaKO5jql7XK_guT1qD73bXJJYFiAQMcFLlke2H0IGlV5E-PiUmPTA0WQRWw'}/ids?startTime={20250720}&endTime={20250723}&type=ranked&start={0}&count={2}'
game_list = requests.get(req_url, headers=REQUEST_HEADERS)

In [28]:
game_list

<Response [200]>

In [19]:
with open('마스터.json','wb') as f:
    json.dump(f)

<Response [504]>

In [4]:
#게임 목록 가져오기
puuid = master_puuid[0]
req_url = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?startTime={20250720}&endTime={20250723}&type=ranked&start={0}&count={20}'
game_list = requests.get(req_url, headers=REQUEST_HEADERS).json()

In [11]:
#특정 게임 가져오기
gameid = 'KR_7723063411'
req_url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{gameid}'
game_info = requests.get(req_url, headers=REQUEST_HEADERS).json()

In [1]:
game_info['info']

NameError: name 'game_info' is not defined

In [20]:
perc

1.0

In [34]:
pings = ['commandPings','getBackPings','holdPings','onMyWayPings', 'allInPings', 'assistMePings','needVisionPings','enemyMissingPings','enemyVisionPings']
ping_count=0
for ping in pings:
    ping_inv_count = game_info['info']['participants'][0][ping]
    if ping_inv_count > 0: ping_count+= ping_inv_count

ping_count

17

In [52]:
#게임 목록 가져오기
gameid = 'KR_7723063411'
req_url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{gameid}/timeline'
game_timeline_info = requests.get(req_url, headers=REQUEST_HEADERS).json()

In [88]:
game_timeline_info['info']['frames'][21]

{'events': [{'killerId': 6,
   'timestamp': 1200622,
   'type': 'WARD_KILL',
   'wardType': 'CONTROL_WARD'},
  {'itemId': 2055,
   'participantId': 7,
   'timestamp': 1205267,
   'type': 'ITEM_DESTROYED'},
  {'creatorId': 7,
   'timestamp': 1205267,
   'type': 'WARD_PLACED',
   'wardType': 'CONTROL_WARD'},
  {'itemId': 1001,
   'participantId': 6,
   'timestamp': 1215694,
   'type': 'ITEM_DESTROYED'},
  {'itemId': 3111,
   'participantId': 6,
   'timestamp': 1215694,
   'type': 'ITEM_PURCHASED'},
  {'itemId': 2055,
   'participantId': 6,
   'timestamp': 1216299,
   'type': 'ITEM_PURCHASED'},
  {'killerId': 5,
   'timestamp': 1217902,
   'type': 'WARD_KILL',
   'wardType': 'CONTROL_WARD'},
  {'creatorId': 10,
   'timestamp': 1218303,
   'type': 'WARD_PLACED',
   'wardType': 'SIGHT_WARD'},
  {'itemId': 2055,
   'participantId': 10,
   'timestamp': 1219071,
   'type': 'ITEM_DESTROYED'},
  {'creatorId': 10,
   'timestamp': 1219071,
   'type': 'WARD_PLACED',
   'wardType': 'CONTROL_WARD'},


In [8]:
#게임 목록 가져오기
req_url = f'https://asia.api.riotgames.com/riot/account/v1/accounts/by-puuid/pz0oSJt_LZhUmkI5PHbn3N309gJWtEy1tT9Jd-D1WiyJY0CsBtJCvLfGqju85O12BDLTNJi-2D1g5w'
game_list = requests.get(req_url, headers=REQUEST_HEADERS).json()

In [9]:
game_list

{'puuid': 'pz0oSJt_LZhUmkI5PHbn3N309gJWtEy1tT9Jd-D1WiyJY0CsBtJCvLfGqju85O12BDLTNJi-2D1g5w',
 'gameName': '이 별',
 'tagLine': 'Gun'}